In [3]:
import json
from datasets import load_dataset
import numpy as np

dataset_id = "phatvo/newsqa"

dataset = load_dataset(dataset_id, data_files="test.csv")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['story_id', 'story_text', 'question', 'answer_token_ranges'],
        num_rows: 5126
    })
})

In [13]:
def _get_text_answer(doc):
    text, _range = doc["story_text"], doc["answer_token_ranges"]
    _range = _range.split(",")[0]
    st, end = [int(each) for each in _range.split(":")]
    tokens = text.split(" ")
    ground_truth = " ".join(tokens[st:end])

    return {"answer": ground_truth}


dataset = dataset.map(_get_text_answer)
dataset["train"][0]

{'story_id': './cnn/stories/289a45e715707cf650352f3eaa123f85d3653d4b.story',
 'story_text': "-LRB- CNN -RRB- -- Comcast rolled out a Web-based on-demand television and movie service on Tuesday that gives customers access to more than 2,000 hours of television and movies . The move comes as users increasingly are bypassing their TV sets and heading straight to the Web -- both legally and illegally -- to watch their favorite shows . The service , named Fancast XFINITY TV -LRB- formerly called TV Everywhere -RRB- , is the biggest cable industry initiative to keep people from skipping traditional TV service in the United States . `` I watch TV online every day . I find it more convenient than my regular TV ... , '' Michael Heard , a self-employed computer repairman from Atlanta , Georgia , said via e-mail . `` I 'm usually watching TV on one window while reading e-mail or tweets on another . `` And also my time is important , so sitting down and watching a show at 8 or 9 p.m. is n't conven

In [30]:
from collections import Counter

unique_contexts = Counter(dataset["train"]["story_text"])
count = 0
raft_train_ctx = []
raft_test_ctx = []
for k, v in unique_contexts.items():
  if v > 5 and count < 100:
    count += 1
    raft_train_ctx.append(k)
  else:
    raft_test_ctx.append(k)

print(len(raft_test_ctx), len(raft_train_ctx))

#raft_train_ctx = unique_contexts[:100]
#raft_test_ctx = unique_contexts[100:]

806 100


In [31]:
test_dataset = dataset["train"].filter(lambda x: x["story_text"] in raft_test_ctx)
test_dataset

Filter: 100%|██████████| 5126/5126 [00:00<00:00, 65078.36 examples/s]


Dataset({
    features: ['story_id', 'story_text', 'question', 'answer_token_ranges', 'answer'],
    num_rows: 4224
})

In [36]:
def save_context_to_file(data:list, filepath):
  with open(filepath, "w") as f:
    f.write("\n".join(data) + "\n")

save_context_to_file(raft_train_ctx, "../data/context_for_generate_newsqa.txt")

In [34]:
# Dataset for evaluation raft

test_dataset.push_to_hub(dataset_id, split="test_raft", private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.95s/it]
c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vohun\.cache\huggingface\hub\datasets--phatvo--newsqa. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/newsqa/commit/a0b07f80b5488dfb459d95dda12391229c59dc71', commit_message='Upload dataset', commit_description='', oid='a0b07f80b5488dfb459d95dda12391229c59dc71', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
"""

python3 raft.py \
    --datapath "data/context_for_generate_newsqa.txt" \
    --output "tmp/newsqa-test-100-raft2" \
    --distractors 3 \
    --doctype txt \
    --chunk_size 512 \
    --questions 1 \
    --p 0.9 \
    --openai_key $OPENAI_KEY \
    --completion_model gpt-4o \
    --splitter breakline

"""

In [ ]:
# $env:OPENAI_KEY=

In [1]:
from datasets import load_dataset

ds = load_dataset("json", data_files="../tmp/newsqa-test-100-raft2.jsonl", )
ds = ds.map(lambda x: {"text": f"{x['instruction']}\nCoT Answer: {x['cot_answer']}"})
ds

Generating train split: 100 examples [00:00, 3655.55 examples/s]
Map: 100%|██████████| 100/100 [00:00<00:00, 2492.47 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 100
    })
})

In [3]:
ds["train"][1]

{'id': 'seed_task_1',
 'type': 'general',
 'question': 'What was the predicted passing distance of the Kosmos 1275 debris from the International Space Station?',
 'context': {'sentences': [["NEW YORK -LRB- CNN -RRB- -- A pedestrian was struck twice by vehicles in Queens and dragged for 17 miles by the second vehicle before police found him dead in Brooklyn , New York City Police Commissioner Ray Kelly told reporters Wednesday . Surveillance video shows a body in the middle of the street . The badly mangled body was discovered under a van after several passing motorists motioned the driver to pull over , Kelly said . Police had not identified the victim Wednesday evening , he said . The first driver called 911 to report he thought he had struck a pedestrian but did not see anyone . It turned out the second driver , in a van , had driven over the man , whose body became lodged under its chassis , according to police . Kelly said the van driver stopped at one point during the drive on New

In [2]:
ds.push_to_hub("phatvo/newsqa-raft-100-p0.9")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.13s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/newsqa-raft-100-p0.9/commit/7a7ac0bb3cc0f38f36751e020f0da88a0ee1a070', commit_message='Upload dataset', commit_description='', oid='7a7ac0bb3cc0f38f36751e020f0da88a0ee1a070', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from datasets import load_dataset

dataset_id = "phatvo/newsqa"
ds = load_dataset(dataset_id, split = "test_raft")